# Multi-Spike Tempotron
An implementation of the multi-spike tempotron in Julia. 
For further details see [Gütig, R. (2016). Spiking neurons can discover predictive features by aggregate-label learning. Science, 351(6277), aab4113.](https://science.sciencemag.org/content/351/6277/aab4113).

## Imports

In [1]:
using Tempotrons
using Tempotrons.InputGen
using Tempotrons.Plots
using Tempotrons.Optimizers
using Plots
using Plots.PlotMeasures;

## Set parameters

In [2]:
N = 10
T = 500
dt = 1
t = collect(0:dt:T)
ν = 3
λ = 0.01
opt = SGD(λ)
n_samples = 10
n_classes = 5
n_steps = 5000
tmp = Tempotron(N = N);

## Generate input samples

In [3]:
base_samples = [[PoissonProcess(ν = ν, T = T)
                 for i = 1:N]
                for j = 1:n_classes]
samples = [(x = [SpikeJitter(s, T = T, σ = 5)
                 for s ∈ base_samples[n_classes*(j - 1)÷n_samples+1]],
            y = n_classes*(j - 1)÷n_samples)
           for j = 1:n_samples];

## Training

In [4]:
# Get the tempotron's output before training
out_b = [tmp(s.x, t = t) for s ∈ samples]

# Get STS before training
θ⃰_b = [GetSTS(tmp, s.x) for s ∈ samples]

# Train the tempotron
@time for i = 1:n_steps
    s = rand(samples)
    Train!(tmp, s.x, s.y, optimizer = opt)
end

# Get the tempotron's output after training
out_a = [tmp(s.x, t = t) for s ∈ samples]

# Get STS after training
θ⃰_a = [GetSTS(tmp, s.x) for s ∈ samples];

  4.633141 seconds (25.07 M allocations: 737.076 MiB, 4.22% gc time)


## Plots

In [5]:
# Prepare to plot
plotlyjs(size = (1000, 1200))
cols = collect(1:n_classes)#palette(:rainbow, n_classes)

# Plots
inp_plots = [PlotInputs(s.x, color = cols[s.y + 1])
             for s ∈ samples]
train_plots = [PlotPotential(tmp, out_b = out_b[i].V, out = out_a[i].V,
                             N_b = length(out_b[i].spikes),
                             N = length(out_a[i].spikes),
                             N_t = samples[i].y,
                             t = t, color = cols[samples[i].y + 1])
                for i = 1:length(samples)]
STS_plots = [PlotSTS(tmp, θ⃰_b = θ⃰_b[i], θ⃰ = θ⃰_a[i],
                     color = cols[samples[i].y + 1])
             for i = 1:length(samples)]
ps = [reshape(inp_plots, 1, :);
      reshape(train_plots, 1, :);
      reshape(STS_plots, 1, :)]
p = plot(ps[:]..., layout = (length(inp_plots), 3), link = :x, left_margin = 8mm)

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5875359963513352445\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15972145809115611834\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5875359963513352445\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

<!DOCTYPE html>
 
 
 Plots.jl